<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/Real_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Instalar las librerías necesarias
!pip install opencv-python
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install torch torchvision

# Descargar los pesos pre-entrenados del modelo SAM
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

# Importar las librerías
import cv2
import numpy as np
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline

# Configurar el modelo SAM
device = "cuda" if torch.cuda.is_available() else "cpu"
model_type = "vit_b"
sam_checkpoint = "sam_vit_b_01ec64.pth"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(sam)

# Subir la imagen a la que deseas aplicar el filtro
print("Por favor, sube la imagen a la que deseas aplicar el filtro:")
uploaded = files.upload()

# Obtener el nombre del archivo subido
image_filename = next(iter(uploaded))
frame = cv2.imread(image_filename)

# Verificar si la imagen se cargó correctamente
if frame is None:
    print("Error al cargar la imagen. Asegúrate de subir una imagen válida.")
else:
    # Convertir la imagen a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Generar las máscaras
    masks = mask_generator.generate(frame_rgb)

    # Seleccionar la máscara más grande
    if len(masks) > 0:
        masks = sorted(masks, key=lambda x: x['area'], reverse=True)
        mask = masks[0]['segmentation']
    else:
        mask = np.zeros(frame.shape[:2], dtype=bool)

    # Crear un filtro simple (dibujar ojos de gato)
    # Aquí creamos un filtro básico usando círculos para simular ojos de gato
    filter_image = np.zeros_like(frame, dtype=np.uint8)

    # Encontrar el contorno de la máscara
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Encontrar los puntos clave (simplificado)
        x, y, w, h = cv2.boundingRect(contours[0])

        # Coordenadas aproximadas para los ojos
        eye_width = int(w * 0.15)
        eye_height = int(h * 0.15)
        left_eye_center = (int(x + w * 0.3), int(y + h * 0.4))
        right_eye_center = (int(x + w * 0.7), int(y + h * 0.4))

        # Dibujar los ojos de gato
        cv2.ellipse(filter_image, left_eye_center, (eye_width, eye_height), 0, 0, 360, (0, 255, 0), -1)
        cv2.ellipse(filter_image, right_eye_center, (eye_width, eye_height), 0, 0, 360, (0, 255, 0), -1)

        # Aplicar el filtro a la imagen original
        alpha = 0.7  # Transparencia del filtro
        mask_expanded = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
        frame = np.where(mask_expanded, cv2.addWeighted(frame, 1 - alpha, filter_image, alpha, 0), frame)

    # Mostrar el resultado
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-9yf83ym5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-9yf83ym5
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
--2024-10-19 00:51:20--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.25, 13.226.210.111, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth.1’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M   136MB/s    in 2.6s    

2024-10-19 00:51:23 (136 MB/s) - ‘sam_vit_b_01ec64.pth.1’ saved [375042383/375042383]

Por

Saving 17292991524401868600684630102405.jpg to 17292991524401868600684630102405.jpg
